In [3]:
%pip install moviepy
%pip install pipwin
%pipwin install pyaudio
%pip install SpeechRecognition
%pip install pydub
%pip install googletrans==4.0.0rc1

  Using cached moviepy-1.0.3.tar.gz (388 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Using cached imageio-2.19.3-py3-none-any.whl (3.4 MB)
  Using cached imageio_ffmpeg-0.4.7-py3-none-win_amd64.whl (22.6 MB)
     ---------------------------------------- 3.3/3.3 MB 35.9 kB/s eta 0:00:00
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110728 sha256=1dee4e1e6f4bce86234eefaf91fd7911ef572416ba06965c553eb0c101a16e93
  Stored in directory: c:\users\tari\appdata\local\pip\cache\wheels\96\32\2d\e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
Note: you may need to restart the kernel to use updated pack

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python310\\Scripts\\imageio_download_bin.exe' -> 'C:\\Python310\\Scripts\\imageio_download_bin.exe.deleteme'



Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%pipwin` not found.


In [1]:
# Import the necessary modules
import os
import time
import wave
import shutil

import pyttsx3
from gtts import gTTS

import numpy as np
import matplotlib.pyplot as plt

import moviepy.editor as mp

import speech_recognition as sr

import googletrans

from pydub import AudioSegment
from pydub.silence import split_on_silence

In [ ]:
# Extract audio from the video
def video_to_audio(video_path, audio_path):
    video = mp.VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

video_to_audio("input/video.mp4", "output/raw_audio.wav")

In [ ]:
def audio_to_text(audio_path, text_path):
    # Read audio file and get the bytes
    audio = wave.open(audio_path, "r")
    audio_wave = audio.readframes(audio.getnframes())

    # Convert to int16
    audio_signal = np.frombuffer(audio_wave, dtype=np.int16)

    # Get time when sound waves were detected using
    audio_time = np.linspace(start=0, stop=len(audio_signal)/audio.getframerate(), num=len(audio_signal))

    # Plot Graph of Audio Signal
    plt.title("Audio Signal")
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.plot(audio_time, audio_signal)
    plt.show()

    # Recognize speech using Google Speech Recognition
    recongnizer = sr.Recognizer()
    recongnizer.energy_threshold = 300

    audio_data = sr.AudioFile(audio_path)
    with audio_data as source:
        recongnizer.adjust_for_ambient_noise(source, duration=3)
        raw = recongnizer.record(source)

        text = recongnizer.recognize_google(raw, language="en-US")

        file = open(text_path, "w")
        file.write(text)
        file.close()

audio_to_text("output/raw_audio.wav", "output/raw_text.txt")


In [ ]:
# This does not work so accurately but good for noisy audio
def audio_to_text(audio_path, text_path):
	# Split the audio file into chunks
	audio = AudioSegment.from_wav(audio_path)
	audio_chunks = split_on_silence(
		audio,
		min_silence_len=1000,
		silence_thresh=-40
	)

	# Store text of each chunk
	raw_text = []

	# Switch to temporary directory
	temp_dir = f"audio_chunks/{time.time_ns()}"
	os.makedirs(temp_dir)
	os.chdir(temp_dir)

	# Process each chunk: Temporary store in a file, recognize speech, store in array,
	for i, chunk in enumerate(audio_chunks):
		print("Converting chunk "+str(i))

		file = f"chunk{str(i)}.wav"
		chunk.export(file, bitrate='192k', format="wav")

		r = sr.Recognizer()
		with sr.AudioFile(file) as source:
			r.adjust_for_ambient_noise(source)
			audio_listened = r.listen(source)

			try:
				text = r.recognize_google(audio_listened)
				raw_text.append(text)
			except sr.UnknownValueError:
				print("Failed to understand audio")
			except sr.RequestError as e:
				print("Could not connect to Internet")

	# Switch back to original directory
	os.chdir("../..")
	shutil.rmtree(temp_dir)

	# Write the text to a file
	file = open(text_path, "w")

	for line in raw_text:
		file.write(line + " ")	

audio_to_text("output/raw_audio.wav", "output/raw_text.txt")

In [5]:
def translate_text(text_path, translated_path, language):
    file = open(text_path, "r")
    raw_text = file.read()
    file.close()

    translator = googletrans.Translator()
    translation = translator.translate(raw_text, dest=language).text

    file = open(translated_path, "w")
    file.write(translation)
    file.close()

translate_text("output/raw_text.txt", "output/translated_text.txt", "es")

In [16]:
def text_to_audio(text_path, audio_path, language):
    # Read the text file
    file = open(text_path, "r")
    raw_text = file.read()
    file.close()

    # Convert text to speech
    tts = gTTS(text=raw_text, lang=language)
    tts.save("audio.wav")
    shutil.move("audio.wav", audio_path)

text_to_audio("output/translated_text.txt", "output/translated_audio.wav", "es")

['audio_chunks', 'input', 'main.ipynb', 'output', 'output_video.mp4']

In [5]:
def change_audio_on_video(video_path, audio_path):
    # Read the video file
    video = mp.VideoFileClip(video_path)
    # Read the audio file
    audio = AudioSegment.from_wav(audio_path)

    # Create a new video file with the audio
    video_with_audio = video.set_audio(audio)
    video_with_audio.write_videofile("output/translated_video.mp4")


change_audio_on_video("input/video.mp4", "output/translated_audio.wav")

FileNotFoundError: [WinError 2] The system cannot find the file specified